# Исследование алгоритмов

### Проверка и установка рабочей директории, должен быть корень проекта

In [1]:
%pwd

'C:\\Users\\Kuroha\\source\\repos_py\\bauman_final_project\\notebooks'

In [2]:
%cd ..

C:\Users\Kuroha\source\repos_py\bauman_final_project


### Загрузка датасета

In [3]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils import *

In [4]:
path = get_filepath(DATA_PROCESSED_TRAIN, is_raw=False)
df = pd.read_csv(path, parse_dates=['date'])

In [5]:
df.head().T

,0,1,2,3,4
uid,0.0,0.0,0.0,0.0,0.0
date,2008-01-01 00:00:00,2008-01-02 00:00:00,2008-01-03 00:00:00,2008-01-04 00:00:00,2008-01-05 00:00:00
latitude,0.312439,0.312439,0.312439,0.312439,0.312439
longitude,0.699807,0.699807,0.699807,0.699807,0.699807
temperature,0.438202,0.314607,0.146067,0.168539,0.325843
cloud,1.0,0.0,0.0,0.666667,1.0
is_fallback_data,1,1,1,1,1
water_level,138.0,138.0,138.0,138.0,138.0
uid_0,1.0,1.0,1.0,1.0,1.0
uid_1,0.0,0.0,0.0,0.0,0.0


### Подготовка данных

Необходимо проверить, зависит ли качество моделей от способа кодирования погоды: одной колонкой с наличием осадков, либо с указанием их вида, двумя - дождь и снег, либо тремя - дождь, гроза и снег.

Датасеты с разные вариантами кодирования погоды будут именоваться как **v1**, **v2**, **v3**, **v4**.

In [6]:
df.columns

Index(['uid', 'date', 'latitude', 'longitude', 'temperature', 'cloud',
       'is_fallback_data', 'water_level', 'uid_0', 'uid_1', 'uid_2', 'uid_3',
       'uid_4', 'uid_5', 'uid_6', 'uid_7', 'uid_8', 'uid_9', 'uid_10',
       'uid_11', 'uid_12', 'uid_13', 'uid_14', 'year', 'day_sin', 'day_cos',
       'weather_v1_precip', 'weather_v2_rain', 'weather_snow',
       'weather_v3_rain', 'weather_v3_storm', 'weather_v4'],
      dtype='object')

Для обучения модели используем данные за 2008-2016 года, а для тестирования - за 2017.

In [7]:
df_X_train = df.loc[(df['date'] < '2017-01-01')]
df_X_test = df.loc[(df['date'] >= '2017-01-01')]
y_train = df_X_train['water_level']
y_test = df_X_test['water_level']

print(f'''Размерность оригинального датасета: {df.shape}
Размерность тренировочного датасета: {df_X_train.shape}
Размерность тестового датасета: {df_X_test.shape}
Размер тестовой выборки: {df_X_test.shape[0] / df_X_train.shape[0] * 100:.2f}%''')

Размерность оригинального датасета: (71159, 32)
Размерность тренировочного датасета: (61535, 32)
Размерность тестового датасета: (9624, 32)
Размер тестовой выборки: 15.64%


In [8]:
def get_all_df(input_df):
    df_all_base = input_df.drop(['date', 'water_level', 'uid'], axis=1)
    df_all_v1 = df_all_base.drop(['weather_v4', 'weather_v2_rain', 'weather_snow', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v2 = df_all_base.drop(['weather_v4', 'weather_v1_precip', 'weather_v3_rain', 'weather_v3_storm'], axis=1)
    df_all_v3 = df_all_base.drop(['weather_v4', 'weather_v1_precip', 'weather_v2_rain'], axis=1)
    
    df_all_v4 = df_all_base.drop(['weather_v2_rain', 'weather_snow', 'weather_v1_precip'], axis=1)
    df_all_v4 = df_all_v4.drop(['weather_v3_rain', 'weather_v3_storm'], axis=1)
    
    return [df_all_v1, df_all_v2, df_all_v3, df_all_v4]

df_X_train_all = get_all_df(df_X_train)
df_X_test_all = get_all_df(df_X_test)

### Применение алгоритмов машинного обучения

Для оценки качества моделей будут использованы 2 меры качества: средняя квадратичная ошибка (MSE) и коэффициент детерминации ($R^2$).

In [9]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

results = {}  # словарь с сохранёнными метриками
algos = {}  # словарь со всеми алгоритмами

In [10]:
# Сохранение модели вместе с оценкой предсказания
def add_result(model, predict, model_name):
    mae_result = round(mae(y_test, predict), 3)
    r2_score_result = round(r2_score(y_test, predict), 3)
    #mae_result = round(mae(y_test, predict), 3)
    #r2_score_result = round(r2_score(y_test, predict), 3)
    print(model_name)
    print(f'MAE: {mae_result}')
    print(f'R2_score: {r2_score_result}')
    print('=' * 20)
    
    results[model_name] = {'MAE': mae_result, 'R2_score': r2_score_result, 'model': model}

# Предсказание по заранее обученной модели
def predict_result(model, X_test, model_name):
    predict = model.predict(X_test)    
    add_result(model, predict, model_name)

# Применение алгоритма ко всем вариациям датасета с последующей оценкой качества
def apply_to_all(algo, model_name):
    if model_name not in algos.keys():
        algos[model_name] = algo
    for i in range(len(df_X_train_all)):
        model = algo(df_X_train_all[i], df_X_test_all[i])    
        predict_result(model, df_X_test_all[i], f'{model_name} (df_v{i+1})')

#### 1. Заглушка

Представим, что модель всегда возвращает средний уровень воды. Полученные ошибки будут использованы для оценки качества реальных моделей.

In [11]:
y_predicted = pd.Series([y_train.mean()] * y_test.shape[0])
y_predicted

0       266.42995
1       266.42995
2       266.42995
3       266.42995
4       266.42995
          ...    
9619    266.42995
9620    266.42995
9621    266.42995
9622    266.42995
9623    266.42995
Length: 9624, dtype: float64

In [12]:
%%time
add_result(None, y_predicted, 'Mean')

Mean
MAE: 153.267
R2_score: -0.014
CPU times: total: 0 ns
Wall time: 1 ms


#### 2. Линейная регрессия

In [13]:
from sklearn.linear_model import LinearRegression 

def linear_regression(X_train, _):    
    linear_regression = LinearRegression()
    linear_regression.fit(X_train, y_train)  
    
    return linear_regression

In [14]:
%%time
apply_to_all(linear_regression, 'Linear Regression')

Linear Regression (df_v1)
MAE: 109.322
R2_score: 0.408
Linear Regression (df_v2)
MAE: 109.304
R2_score: 0.408
Linear Regression (df_v3)
MAE: 109.31
R2_score: 0.409
Linear Regression (df_v4)
MAE: 109.318
R2_score: 0.408
CPU times: total: 562 ms
Wall time: 110 ms


In [15]:
pd.DataFrame.from_dict(results, orient='index')

,MAE,R2_score,model
Mean,153.267,-0.014,None
Linear Regression (df_v1),109.322,0.408,LinearRegression()
Linear Regression (df_v2),109.304,0.408,LinearRegression()
Linear Regression (df_v3),109.310,0.409,LinearRegression()
Linear Regression (df_v4),109.318,0.408,LinearRegression()


#### 3. Дерево решений

In [16]:
from sklearn.tree import DecisionTreeRegressor

def decision_tree(X_train, _):
    decision_tree = DecisionTreeRegressor()
    decision_tree.fit(X_train, y_train)
    
    return decision_tree

In [17]:
%%time
apply_to_all(decision_tree, 'Decision Tree')

Decision Tree (df_v1)
MAE: 70.391
R2_score: 0.593
Decision Tree (df_v2)
MAE: 70.527
R2_score: 0.593
Decision Tree (df_v3)
MAE: 70.464
R2_score: 0.591
Decision Tree (df_v4)
MAE: 70.398
R2_score: 0.592
CPU times: total: 1.64 s
Wall time: 941 ms


#### 4. Гребневая регрессия

In [18]:
from sklearn.linear_model import Ridge

def ridge(X_train, _):
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    
    return ridge

In [19]:
%%time
apply_to_all(ridge, 'Ridge')

Ridge (df_v1)
MAE: 109.323
R2_score: 0.408
Ridge (df_v2)
MAE: 109.305
R2_score: 0.408
Ridge (df_v3)
MAE: 109.311
R2_score: 0.409
Ridge (df_v4)
MAE: 109.319
R2_score: 0.408
CPU times: total: 297 ms
Wall time: 57.1 ms


#### 5. Лассо

In [20]:
from sklearn.linear_model import Lasso

def lasso(X_train, _):
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    
    return lasso

In [21]:
%%time
apply_to_all(lasso, 'Lasso')

Lasso (df_v1)
MAE: 118.103
R2_score: 0.364
Lasso (df_v2)
MAE: 118.103
R2_score: 0.364
Lasso (df_v3)
MAE: 118.103
R2_score: 0.364
Lasso (df_v4)
MAE: 118.175
R2_score: 0.364
CPU times: total: 562 ms
Wall time: 84.3 ms


#### 6. Простая нейронная сеть

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import mean_absolute_error


def simple_neural_network(X_train, X_test):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=[X_train.shape[1]]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    
    model.compile(loss=mean_absolute_error, optimizer='Adam', metrics=['mae'])
    model.fit(X_train, y_train,
              validation_data=(X_test, y_test),
              epochs=10,
              verbose=0)
    
    return model

In [23]:
%%time
apply_to_all(simple_neural_network, 'Neural Network (simple)')

301/301 [==============================] - 0s 661us/step
Neural Network (simple) (df_v1)
MAE: 66.649
R2_score: 0.633
301/301 [==============================] - 0s 677us/step
Neural Network (simple) (df_v2)
MAE: 66.166
R2_score: 0.597
301/301 [==============================] - 0s 678us/step
Neural Network (simple) (df_v3)
MAE: 71.26
R2_score: 0.618
301/301 [==============================] - 0s 705us/step
Neural Network (simple) (df_v4)
MAE: 66.598
R2_score: 0.612
CPU times: total: 17min 6s
Wall time: 1min 36s


#### 7. Нейронная сеть с ранней остановкой

In [24]:
from tensorflow.keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(
    min_delta=0.1,
    patience=20,
    restore_best_weights=True,
)

def neural_network_with_ES(X_train, X_test):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=[X_train.shape[1]]))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    
    model.compile(loss=mean_absolute_error, optimizer='Adam', metrics=['mae'])
    model.fit(X_train, y_train,
              validation_data=(X_test, y_test),
              epochs=100,
              callbacks=[early_stopping],
              verbose=0)
    
    return model

In [25]:
%%time
apply_to_all(neural_network_with_ES, 'Neural Network (early stopping)')

301/301 [==============================] - 0s 654us/step
Neural Network (early stopping) (df_v1)
MAE: 63.693
R2_score: 0.639
301/301 [==============================] - 0s 677us/step
Neural Network (early stopping) (df_v2)
MAE: 65.076
R2_score: 0.648
301/301 [==============================] - 0s 672us/step
Neural Network (early stopping) (df_v3)
MAE: 63.727
R2_score: 0.655
301/301 [==============================] - 0s 644us/step
Neural Network (early stopping) (df_v4)
MAE: 63.829
R2_score: 0.623
CPU times: total: 55min 4s
Wall time: 5min 6s


#### 8. Нейронная сеть со слоями Batch Normalization и Dropout

In [26]:
from tensorflow.keras.layers import BatchNormalization, Dropout


def neural_network_with_Dropout(X_train, X_test):
    model = Sequential()
    model.add(Dense(1024, activation='relu', input_shape=[X_train.shape[1]]))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1))
    
    model.compile(loss=mean_absolute_error, optimizer='Adam', metrics=['mae'])
    model.fit(X_train, y_train,
              validation_data=(X_test, y_test),
              epochs=100,
              callbacks=[early_stopping],
              verbose=0)
    
    return model

In [27]:
%%time
apply_to_all(neural_network_with_Dropout, 'Neural Network (Dropout)')

301/301 [==============================] - 1s 2ms/step
Neural Network (Dropout) (df_v1)
MAE: 64.753
R2_score: 0.543
301/301 [==============================] - 1s 2ms/step
Neural Network (Dropout) (df_v2)
MAE: 63.658
R2_score: 0.577
301/301 [==============================] - 1s 2ms/step
Neural Network (Dropout) (df_v3)
MAE: 62.79
R2_score: 0.593
301/301 [==============================] - 1s 2ms/step
Neural Network (Dropout) (df_v4)
MAE: 59.287
R2_score: 0.589
CPU times: total: 3h 39min 35s
Wall time: 19min 40s


#### 9. Эластичная сеть

In [28]:
from sklearn.linear_model import ElasticNet

def elastic_net(X_train, _):
    elastic_net = ElasticNet()
    elastic_net.fit(X_train, y_train)
    
    return elastic_net

In [29]:
%%time
apply_to_all(elastic_net, 'Elastic Net')

Elastic Net (df_v1)
MAE: 141.898
R2_score: 0.142
Elastic Net (df_v2)
MAE: 141.902
R2_score: 0.142
Elastic Net (df_v3)
MAE: 141.902
R2_score: 0.142
Elastic Net (df_v4)
MAE: 141.914
R2_score: 0.142
CPU times: total: 1.03 s
Wall time: 87.1 ms


#### 10. Регрессия опорных векторов

In [30]:
from sklearn.svm import SVR

def svr(X_train, _):
    svr = SVR()
    svr.fit(X_train, y_train)
    
    return svr

In [31]:
%%time
apply_to_all(svr, 'SVR')

SVR (df_v1)
MAE: 83.632
R2_score: 0.416
SVR (df_v2)
MAE: 83.78
R2_score: 0.415
SVR (df_v3)
MAE: 83.833
R2_score: 0.414
SVR (df_v4)
MAE: 85.926
R2_score: 0.394
CPU times: total: 19min 42s
Wall time: 19min 42s


#### 11. Градиентный бустинг

In [32]:
from sklearn.ensemble import GradientBoostingRegressor

def gbr(X_train, _):
    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, y_train)
    
    return gbr

In [33]:
%%time
apply_to_all(gbr, 'Gradient Boosting')

Gradient Boosting (df_v1)
MAE: 66.847
R2_score: 0.682
Gradient Boosting (df_v2)
MAE: 66.847
R2_score: 0.682
Gradient Boosting (df_v3)
MAE: 66.847
R2_score: 0.682
Gradient Boosting (df_v4)
MAE: 66.847
R2_score: 0.682
CPU times: total: 17.9 s
Wall time: 17.9 s


#### 12. Случайный лес 

In [34]:
from sklearn.ensemble import RandomForestRegressor

def random_forest(X_train, _):
    random_forest = RandomForestRegressor()
    random_forest.fit(X_train, y_train)
    
    return random_forest

In [35]:
%%time
apply_to_all(random_forest, 'Random Forest')

Random Forest (df_v1)
MAE: 68.282
R2_score: 0.628
Random Forest (df_v2)
MAE: 68.337
R2_score: 0.627
Random Forest (df_v3)
MAE: 68.481
R2_score: 0.624
Random Forest (df_v4)
MAE: 68.353
R2_score: 0.626
CPU times: total: 1min
Wall time: 1min


#### 13. XGBoost

In [36]:
from xgboost.sklearn import XGBRegressor


def xgboost(X_train, _):
    xgboost = XGBRegressor()
    xgboost.fit(X_train, y_train)
    
    return xgboost

In [37]:
%%time
apply_to_all(xgboost, 'XGBoost')

XGBoost (df_v1)
MAE: 66.044
R2_score: 0.653
XGBoost (df_v2)
MAE: 66.342
R2_score: 0.65
XGBoost (df_v3)
MAE: 66.391
R2_score: 0.648
XGBoost (df_v4)
MAE: 65.685
R2_score: 0.655
CPU times: total: 44.7 s
Wall time: 3.75 s


#### 14. Настроенный XGBoost

Поиск лучших гиперпараметров через GridSearchCV возможно улучшит работу алгоритма XGBoost.

In [38]:
from sklearn.model_selection import GridSearchCV

X_train = df_X_train_all[0]
parameters = {
              'gamma':[0.0, 0.025, 0.05, 0.075],  # 0
              'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.3],  # 0.3
              'max_depth':[6, 7, 8, 9],  # default=6
              'min_child_weight': [0, 1]  # 1
             }

In [39]:
xgboost_tuned = XGBRegressor()
xgb_grid = GridSearchCV(xgboost_tuned, parameters, cv = 2, n_jobs = 5, verbose=True, 
                        scoring='r2')
#xgb_grid.fit(X_train, y_train)
#xgb_grid.best_params_
print("{'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 0}")

{'gamma': 0.0, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 0}


In [40]:
def xgboost_tuned(X_train, _):
    xgboost_tuned = XGBRegressor(learning_rate=0.05, min_child_weight=0)
    xgboost_tuned.fit(X_train, y_train)
    
    return xgboost_tuned

In [41]:
%%time
apply_to_all(xgboost_tuned, 'XGBoost (Tuned)')

XGBoost (Tuned) (df_v1)
MAE: 63.877
R2_score: 0.678
XGBoost (Tuned) (df_v2)
MAE: 63.877
R2_score: 0.678
XGBoost (Tuned) (df_v3)
MAE: 63.757
R2_score: 0.679
XGBoost (Tuned) (df_v4)
MAE: 63.876
R2_score: 0.678
CPU times: total: 44 s
Wall time: 3.67 s


In [42]:
frame = pd.DataFrame.from_dict(results, orient='index')
frame.sort_values(by='R2_score', ascending=False)

,MAE,R2_score,model
Gradient Boosting (df_v2),66.847,0.682,([DecisionTreeRegressor(criterion='friedman_ms...
Gradient Boosting (df_v1),66.847,0.682,([DecisionTreeRegressor(criterion='friedman_ms...
Gradient Boosting (df_v3),66.847,0.682,([DecisionTreeRegressor(criterion='friedman_ms...
Gradient Boosting (df_v4),66.847,0.682,([DecisionTreeRegressor(criterion='friedman_ms...
XGBoost (Tuned) (df_v3),63.757,0.679,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (Tuned) (df_v4),63.876,0.678,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (Tuned) (df_v2),63.877,0.678,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (Tuned) (df_v1),63.877,0.678,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (df_v4),65.685,0.655,"XGBRegressor(base_score=0.5, booster='gbtree',..."
Neural Network (early stopping) (df_v3),63.727,0.655,<keras.engine.sequential.Sequential object at ...


Наиболее качественные прогнозы дают алгоритмы **Gradient Boosting**, **XGBoost** и **нейронная сеть с ранней остановкой**.

Кодирование погоды 3-им (дождь, гроза и снег) способами показывают схожие, либо лучшие результаты относительно кодирования остальными способами. В дальнейшем погода будет кодироваться именно таким образом.
***
Попробуем использовать те же алгоритмы, но на датасетах с **третьим** способом кодирования погоды, в которых используется целочисленное значение uid, без индивидуальных столбцов (**v1**), либо убраны данные об облачности (**v2**), либо всё сразу (**v3**).

In [43]:
def get_all_df(input_df):
    df_all_base = input_df.drop(['date', 'water_level', 'weather_v4', 'weather_v1_precip', 'weather_v4'], axis=1)
    df_all_base = df_all_base.drop(['weather_v2_rain'], axis=1)
    
    uid_cols = [f'uid_{i}' for i in range(0, 15)]
    
    df_all_v1 = df_all_base.drop(uid_cols, axis=1)
    df_all_v2 = df_all_base.drop(['uid', 'cloud'], axis=1)
    df_all_v3 = df_all_base.drop(uid_cols + ['cloud'], axis=1)
    return [df_all_v1, df_all_v2, df_all_v3]

df_X_train_all = get_all_df(df_X_train)
df_X_test_all = get_all_df(df_X_test)

results_weather = results.copy()
results = {}  # очищение списка результатов

In [44]:
for name, algo in algos.items():
    apply_to_all(algo, name)
    print('*' * 25)

Linear Regression (df_v1)
MAE: 122.497
R2_score: 0.339
Linear Regression (df_v2)
MAE: 108.495
R2_score: 0.41
Linear Regression (df_v3)
MAE: 121.528
R2_score: 0.34
*************************
Decision Tree (df_v1)
MAE: 55.019
R2_score: 0.75
Decision Tree (df_v2)
MAE: 70.473
R2_score: 0.592
Decision Tree (df_v3)
MAE: 54.839
R2_score: 0.752
*************************
Ridge (df_v1)
MAE: 122.496
R2_score: 0.339
Ridge (df_v2)
MAE: 108.497
R2_score: 0.41
Ridge (df_v3)
MAE: 121.527
R2_score: 0.34
*************************
Lasso (df_v1)
MAE: 125.044
R2_score: 0.327
Lasso (df_v2)
MAE: 117.08
R2_score: 0.367
Lasso (df_v3)
MAE: 123.975
R2_score: 0.33
*************************
301/301 [==============================] - 0s 696us/step
Neural Network (simple) (df_v1)
MAE: 74.881
R2_score: 0.675
301/301 [==============================] - 0s 705us/step
Neural Network (simple) (df_v2)
MAE: 67.56
R2_score: 0.662
301/301 [==============================] - 0s 647us/step
Neural Network (simple) (df_v3)
MAE: 74.

In [45]:
frame = pd.DataFrame.from_dict(results, orient='index')
frame.sort_values(by='R2_score', ascending=False)

,MAE,R2_score,model
XGBoost (Tuned) (df_v3),47.915,0.849,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (Tuned) (df_v1),47.910,0.849,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (df_v1),50.841,0.820,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (df_v3),50.787,0.820,"XGBRegressor(base_score=0.5, booster='gbtree',..."
Gradient Boosting (df_v3),58.972,0.793,([DecisionTreeRegressor(criterion='friedman_ms...
Gradient Boosting (df_v1),58.972,0.793,([DecisionTreeRegressor(criterion='friedman_ms...
Random Forest (df_v3),53.153,0.779,"(DecisionTreeRegressor(max_features=1.0, rando..."
Random Forest (df_v1),53.082,0.778,"(DecisionTreeRegressor(max_features=1.0, rando..."
Neural Network (Dropout) (df_v3),52.830,0.769,<keras.engine.sequential.Sequential object at ...
Neural Network (Dropout) (df_v1),54.768,0.763,<keras.engine.sequential.Sequential object at ...


В данном эксперимента, наиболее качественные прогнозы дают алгоритмы **XGBoost**, **Gradient Boosting** и **Random Forest**.

Самый лучший результат у **настроенного XGBoost**, у которого ошибки **47.915** и **0.849**.

Качество прогноза заметно повышается на некоторых алгоритмах, когда для кодирования uid используется одно число. Отсутствие данных об облачности незначительно влияет на предсказания. В дальнейшем из датасетов будут исключаться данные об облачности (**cloud**).
***
Теперь попробуем использовать алгоритмы на датасетах без синуса (**v1**) или косинуса (**v2**) дня от года как с числовым представлением uid, так и с индивидуальными столбцами (**v3** и **v4**).

In [47]:
def get_all_df(input_df):
    df_all_base = input_df.drop(['date', 'water_level', 'cloud', 'weather_v1_precip', 'weather_v2_rain', 'weather_v4'], axis=1)
    
    uid_cols = [f'uid_{i}' for i in range(0, 15)]
    df_all_v1 = df_all_base.drop(uid_cols + ['day_sin'], axis=1)
    df_all_v2 = df_all_base.drop(uid_cols + ['day_cos'], axis=1)
    
    df_all_v3 = df_all_base.drop(['uid', 'day_sin'], axis=1)
    df_all_v4 = df_all_base.drop(['uid', 'day_cos'], axis=1)
    
    return [df_all_v1, df_all_v2, df_all_v3, df_all_v4]

df_X_train_all = get_all_df(df_X_train)
df_X_test_all = get_all_df(df_X_test)

results_uids_cloud = results.copy()
results = {}  # очищение списка результатов

In [48]:
for name, algo in algos.items():
    apply_to_all(algo, name)
    print('*' * 25)

Linear Regression (df_v1)
MAE: 122.233
R2_score: 0.335
Linear Regression (df_v2)
MAE: 121.415
R2_score: 0.312
Linear Regression (df_v3)
MAE: 108.26
R2_score: 0.406
Linear Regression (df_v4)
MAE: 108.556
R2_score: 0.38
*************************
Decision Tree (df_v1)
MAE: 70.098
R2_score: 0.522
Decision Tree (df_v2)
MAE: 56.133
R2_score: 0.755
Decision Tree (df_v3)
MAE: 83.775
R2_score: 0.385
Decision Tree (df_v4)
MAE: 72.061
R2_score: 0.598
*************************
Ridge (df_v1)
MAE: 122.23
R2_score: 0.335
Ridge (df_v2)
MAE: 121.415
R2_score: 0.312
Ridge (df_v3)
MAE: 108.257
R2_score: 0.406
Ridge (df_v4)
MAE: 108.554
R2_score: 0.38
*************************
Lasso (df_v1)
MAE: 124.243
R2_score: 0.324
Lasso (df_v2)
MAE: 123.403
R2_score: 0.308
Lasso (df_v3)
MAE: 116.588
R2_score: 0.362
Lasso (df_v4)
MAE: 115.801
R2_score: 0.345
*************************
301/301 [==============================] - 0s 679us/step
Neural Network (simple) (df_v1)
MAE: 85.504
R2_score: 0.52
301/301 [===========

In [49]:
frame = pd.DataFrame.from_dict(results, orient='index')
frame.sort_values(by='R2_score', ascending=False)

,MAE,R2_score,model
XGBoost (Tuned) (df_v2),51.172,0.821,"XGBRegressor(base_score=0.5, booster='gbtree',..."
XGBoost (df_v2),52.378,0.807,"XGBRegressor(base_score=0.5, booster='gbtree',..."
Random Forest (df_v2),54.010,0.789,"(DecisionTreeRegressor(max_features=1.0, rando..."
Neural Network (early stopping) (df_v2),57.284,0.756,<keras.engine.sequential.Sequential object at ...
Decision Tree (df_v2),56.133,0.755,DecisionTreeRegressor()
XGBoost (Tuned) (df_v1),58.852,0.748,"XGBRegressor(base_score=0.5, booster='gbtree',..."
Neural Network (Dropout) (df_v2),54.099,0.732,<keras.engine.sequential.Sequential object at ...
Gradient Boosting (df_v2),65.146,0.728,([DecisionTreeRegressor(criterion='friedman_ms...
XGBoost (df_v1),59.101,0.726,"XGBRegressor(base_score=0.5, booster='gbtree',..."
Gradient Boosting (df_v1),69.141,0.700,([DecisionTreeRegressor(criterion='friedman_ms...


Отсутствие столбца с косинусом дня положительно сказалось на алгоритме **Random Forest**, однако лучший результат был получен в прошлом эксперименте, а именно **настроенный XGBoost** с ошибками **47.915** и **0.849**, для работы которого необходимы следующие данные:
- числовой uid
- широта и долгота
- температура
- is_fallback_data (был использован прогноз погоды из соседней локации)
- год
- день в году, закодированный синусом и косинусом
- погода тремя столбцами: снег, дождь и шторм

Обучим данный алгоритм на всех данных и сохраним его.

In [57]:
xgboost_tuned = results_uids_cloud['XGBoost (Tuned) (df_v3)']['model']
xgboost_tuned

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=0, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [56]:
df.head().columns

Index(['uid', 'date', 'latitude', 'longitude', 'temperature', 'cloud',
       'is_fallback_data', 'water_level', 'uid_0', 'uid_1', 'uid_2', 'uid_3',
       'uid_4', 'uid_5', 'uid_6', 'uid_7', 'uid_8', 'uid_9', 'uid_10',
       'uid_11', 'uid_12', 'uid_13', 'uid_14', 'year', 'day_sin', 'day_cos',
       'weather_v1_precip', 'weather_v2_rain', 'weather_snow',
       'weather_v3_rain', 'weather_v3_storm', 'weather_v4'],
      dtype='object')

In [58]:
X = df[['uid', 'latitude', 'longitude', 'temperature', 'is_fallback_data', 'year', 
        'day_sin', 'day_cos', 'weather_snow', 'weather_v3_rain', 'weather_v3_storm']]
y = df['water_level']

0        138.0
1        138.0
2        138.0
3        138.0
4        138.0
         ...  
71154     88.0
71155     88.0
71156     89.0
71157     89.0
71158     89.0
Name: water_level, Length: 71159, dtype: float64

In [63]:
xgboost_tuned.fit(X, y)  # повторный запуск обучения сбросит ранее вычисленные веса

#model_path = os.path.join(os.getcwd(), 'models', 'xgboost_model.json')
model_path = get_xgboost_path()
xgboost_tuned.save_model(model_path)

In [67]:
predict = xgboost_tuned.predict(X)
mae_result = round(mae(y, predict), 3)
r2_score_result = round(r2_score(y, predict), 3)
print(f'MAE: {mae_result}')
print(f'R2_score: {r2_score_result}')

MAE: 35.24
R2_score: 0.923


После сохранения модели необходимо проверить её работоспособность:

In [69]:
xgboost = XGBRegressor()
xgboost.load_model(model_path)

predict = xgboost.predict(X)
mae_result = round(mae(y, predict), 3)
r2_score_result = round(r2_score(y, predict), 3)
print(f'MAE: {mae_result}')
print(f'R2_score: {r2_score_result}')

MAE: 35.24
R2_score: 0.923


После сохранения модели она показывает те же результаты, что и до сохранения.